In [23]:
# Download zip directly
!wget -cq https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-Recurrent-Neural-Networks.zip
!unzip -qq P16-Recurrent-Neural-Networks.zip

replace Recurrent_Neural_Networks/Google_Stock_Price_Test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Recurrent_Neural_Networks/Google_Stock_Price_Train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace Recurrent_Neural_Networks/rnn.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace Recurrent_Neural_Networks/rnn.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [0]:
home_path = '/content/Recurrent_Neural_Networks'

# Imports

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf 
from keras import backend as K
from tensorflow.python.client import device_lib

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

# Set up GPU session

In [26]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13164987640351932617
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2798876207290315216
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11304609345699470563
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6746195743447841485
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [27]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

# Part 1 - Data Preprocessing


In [28]:
# Importing the training set
dataset_train = pd.read_csv(home_path+'/Google_Stock_Price_Train.csv')
dataset_train.head()

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"
3,1/6/2012,328.34,328.77,323.68,648.24,"5,405,900"
4,1/9/2012,322.04,322.29,309.46,620.76,"11,688,800"


In [29]:
training_set = dataset_train.iloc[:, 1:2].values
print(training_set.shape)

(1258, 1)


In [0]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [31]:
# Creating a data structure with 60 timesteps and 1 output (alternative to the cell below)
steps = 60

X_train = np.array([training_set_scaled[i-steps:i, 0] for i in range(steps, training_set.shape[0])])
y_train = np.array([training_set_scaled[i, 0] for i in range(steps, training_set.shape[0])])

print(X_train)
print(X_train.shape, y_train.shape)

[[0.08581368 0.09701243 0.09433366 ... 0.07846566 0.08034452 0.08497656]
 [0.09701243 0.09433366 0.09156187 ... 0.08034452 0.08497656 0.08627874]
 [0.09433366 0.09156187 0.07984225 ... 0.08497656 0.08627874 0.08471612]
 ...
 [0.92106928 0.92438053 0.93048218 ... 0.95475854 0.95204256 0.95163331]
 [0.92438053 0.93048218 0.9299055  ... 0.95204256 0.95163331 0.95725128]
 [0.93048218 0.9299055  0.93113327 ... 0.95163331 0.95725128 0.93796041]]
(1198, 60) (1198,)


In [32]:
# Creating a data structure with 60 timesteps and 1 output
steps = 60
X_train = []
y_train = []

for i in range(steps, training_set.shape[0]):
    X_train.append(training_set_scaled[i-steps:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train)
print(X_train.shape, y_train.shape)

[[0.08581368 0.09701243 0.09433366 ... 0.07846566 0.08034452 0.08497656]
 [0.09701243 0.09433366 0.09156187 ... 0.08034452 0.08497656 0.08627874]
 [0.09433366 0.09156187 0.07984225 ... 0.08497656 0.08627874 0.08471612]
 ...
 [0.92106928 0.92438053 0.93048218 ... 0.95475854 0.95204256 0.95163331]
 [0.92438053 0.93048218 0.9299055  ... 0.95204256 0.95163331 0.95725128]
 [0.93048218 0.9299055  0.93113327 ... 0.95163331 0.95725128 0.93796041]]
(1198, 60) (1198,)


In [33]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_train.shape

(1198, 60, 1)



# Part 2 - Building the RNN


In [0]:
# For input_shape, only features (past stock prices) 
# and predictors (could be other similar stocks like Apple etc.) are needed

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# USUALLY RNNs use the optimizer Rmsprop, but adam is safe to start with!                    !!!!!!
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [35]:
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
1198/1198 [==============================] - 22s 18ms/sample - loss: 0.0494
Epoch 2/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0064
Epoch 3/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0050
Epoch 4/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0055
Epoch 5/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0054
Epoch 6/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0041
Epoch 7/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0045
Epoch 8/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0040
Epoch 9/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0044
Epoch 10/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0043
Epoch 11/100
1198/1198 [==============================] - 19s 16ms/sample - loss: 0.0037
Epoch 12/100
1198/1198 [======



# Part 3 - Making the predictions and visualising the results


In [0]:
# Getting the real stock price of 2017
dataset_test = pd.read_csv(home_path+'Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

In [0]:
# Getting the predicted stock price of 2017
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []

for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
    
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()